## Important Dependencies with their required versions


In [ ]:

# ! pip install SQLAlchemy==2.0.26
# ! pip install langchain==0.1.6
# ! pip install langchain-community==0.0.19
# ! pip install langchain-core==0.1.22
# ! pip install chromadb
# !pip install text_generation

## Imports and Setup and Initializing Agent

In [1]:
# # to auto reload instead of having to restart each time
%load_ext autoreload
%autoreload 2

In [ ]:
# from langchain_community.llms import HuggingFaceEndpoint, HuggingFaceHub
# import os
# from dotenv import load_dotenv
# from langchain.chains import LLMChain
# from langchain.prompts import PromptTemplate

# question = "Who won the FIFA World Cup in the year 1994? "

# template = """Question: {question}

# Answer: Let's think step by step."""

# prompt = PromptTemplate.from_template(template)

# repo_id = "bigcode/starcoder2-7b"
# load_dotenv()
# # llm = HuggingFaceEndpoint(
# #     repo_id=repo_id, max_new_tokens=12000, temperature=0.5, token= os.environ.get("HUGGINGFACE_TOKEN")
# # )

# llm = HuggingFaceHub(
#         repo_id=repo_id,
#         huggingfacehub_api_token= os.environ.get("HUGGINGFACE_TOKEN"),
#         task="text-generation",
#         model_kwargs={
#             "temperature": 0.1,  # can't be 0
#             "max_new_tokens": 12000,
#         },
#         cache=False,
#     )

# llm_chain = LLMChain(prompt=prompt, llm=llm)
# print(llm_chain.run({"question":question}))

In [1]:
# from Imports import *
try:
    from Configuration import *
    from TestGenerator import *
    from TestFix import *

    llm_arb, chat_model_arb = InitializeModelArbiter(os.environ["HUGGINGFACEHUB_API_TOKEN"])

    t = TestGenerator(GenUnitTestChain, db, HEval_JsonObj, globals())
    f = TestFix(globals(), chat_model)
except Exception as e:
    print(e)
    exit(-1)

C:\Users\omare\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.huggingface_hub.HuggingFaceHub` was deprecated in langchain-community 0.0.21 and will be removed in 0.2.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(
C:\Users\omare\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.huggingface_text_gen_inference.HuggingFaceTextGenInference` was deprecated in langchain-community 0.0.21 and will be removed in 0.2.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(
WARNING! repo_id is not default parameter.
                    repo_id was transferred to model_kwargs.
                    Please confirm that repo_id is what you in

File ResultsEachTest/Mixtral-3Shot/RunningLogs.json does not exist


## Generate Unit Tests from Code

In [2]:
from TestGenerator import *
from TestFix import *

llm_arb, chat_model_arb = InitializeModelArbiter(os.environ["HUGGINGFACEHUB_API_TOKEN"])

# t = TestGenerator(GenUnitTestChain, db, HEval_JsonObj, globals())
f = TestFix(globals(), chat_model)

WARNING! repo_id is not default parameter.
                    repo_id was transferred to model_kwargs.
                    Please confirm that repo_id is what you intended.
WARNING! task is not default parameter.
                    task was transferred to model_kwargs.
                    Please confirm that task is what you intended.
WARNING! huggingfacehub_api_token is not default parameter.
                    huggingfacehub_api_token was transferred to model_kwargs.
                    Please confirm that huggingfacehub_api_token is what you intended.


FileNotFoundError: File ResultsEachTest/Mixtral-3Shot/RunningLogs.json does not exist

In [ ]:
llm("What is 4 + 5?")

In [2]:
t.generate()

File OutputTest/Cases.json created successfully!
File OutputTest/RunningLogs.json created successfully!
Running Test Case  0

Test example 0 succeeded

Number of Ran Tests :  6
Number of Succeeded Test :  6
Number of Succeeded Test :  0
Running Test Case  1
ERROR: test_separate_paren_groups_invalid_string (__main__.SeparateParenGroupsTest)
Traceback (most recent call last):
  File "<string>", line 50, in test_separate_paren_groups_invalid_string
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\unittest\case.py", line 738, in assertRaises
    return context.handle('assertRaises', args, kwargs)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\unittest\case.py", line 201, in handle
    callable_obj(*args, **kwargs)
TypeError: 'list' object is not callable

FAIL: test_separate_paren_groups_nested_groups (__main__.SeparateParenGroupsTest)
Traceback (most recent call last):

In [3]:
f.generate()

Directory FeedbackOutput/ created successfully!
File FeedbackOutput/Cases.json created successfully!
File FeedbackOutput/RunningLogs.json created successfully!
Running Example  0 

<s>[INST] 
you generated unittest for the following method under test having the following description:
*Method under test*:

from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    for idx, elem in enumerate(numbers):
        for idx2, elem2 in enumerate(numbers):
            if idx != idx2:
                distance = abs(elem - elem2)
                if distance < threshold:
                    return True

    return False


*Description*:
    Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True


but this test failed
*Test*:
    def test_has_close_elements_first_and_last_close(self